In [ ]:
!pip install keras==2.1.6

     |████████████████████████████████| 348kB 8.7MB/s 
  Found existing installation: Keras 2.2.4
    Uninstalling Keras-2.2.4:
      Successfully uninstalled Keras-2.2.4


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
from keras.preprocessing.image import img_to_array, load_img
import os
from keras import models,layers,optimizers
from keras.applications import VGG16,Xception
from sklearn.model_selection import train_test_split
import numpy as np
import tensorflow as tf
from tensorflow.python.keras import utils
from keras.utils import np_utils
import cv2
import itertools
from keras.models import load_model

input_size = 224

Using TensorFlow backend.


Run this for creating model for Stationary Signs [A-I,K-Y]

In [ ]:
path = "/content/gdrive/My Drive/StationarySigns"
noOfClasses = 25
classes = {
    
    'A': 0,
    'B': 1,
    'C': 2,
    'D': 3,
    'E': 4,
    'F': 5,
    'G': 6,
    'H': 7,
    'I': 8,
    'K': 9,
    'L': 10,
    'M': 11,
    'N': 12,
    'O': 13,
    'P': 14,
    'Q': 15,
    'R': 16,
    'S': 17,
    'T': 18,
    'U': 19,
    'V': 20,
    'W': 21,
    'X': 22,
    'Y': 23,
    '^': 24
    
}

Run this for Motion Signs [J,Z]

In [ ]:
path = "/content/gdrive/My Drive/MotionSigns"
noOfClasses = 3
classes = {
    'J': 0,
    'Z': 1,
    '^': 2
    
}

In [ ]:
def load_data():
  X = []
  y = []
  for root, directories, filenames in os.walk(path):
    print(root)
    for filename in filenames:
      if filename.endswith(".jpg"):
        if int(filename[2:-4])<=200:
          fullpath = os.path.join(root, filename)
          img = load_img(fullpath)
          img = img_to_array(img)
          img = cv2.resize(img,(224,224))
          X.append(img)
          t = fullpath.rindex('/')
          fullpath = fullpath[0:t]
          n = fullpath.rindex('/')
          y.append(classes[fullpath[n + 1:t]])
  y = np.asarray(y)
  y = y.reshape(y.shape[0], 1)
  X = np.asarray(X).astype('float32')
  X = X / 255.0
  y = np_utils.to_categorical(y, noOfClasses)
  
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 42)
  np.save(path+"/X_train.npy",X_train)
  np.save(path+"/X_test.npy",X_test)
  np.save(path+"/y_train.npy",y_train)
  np.save(path+"/y_test.npy",y_test)
  print("Data loaded!!!")
load_data()

/content/gdrive/My Drive/dataset_final_3
/content/gdrive/My Drive/dataset_final_3/Y
/content/gdrive/My Drive/dataset_final_3/V
/content/gdrive/My Drive/dataset_final_3/R
/content/gdrive/My Drive/dataset_final_3/U
/content/gdrive/My Drive/dataset_final_3/X
/content/gdrive/My Drive/dataset_final_3/L
/content/gdrive/My Drive/dataset_final_3/K
/content/gdrive/My Drive/dataset_final_3/O
/content/gdrive/My Drive/dataset_final_3/H
/content/gdrive/My Drive/dataset_final_3/Q
/content/gdrive/My Drive/dataset_final_3/P
/content/gdrive/My Drive/dataset_final_3/I
/content/gdrive/My Drive/dataset_final_3/G
/content/gdrive/My Drive/dataset_final_3/F
/content/gdrive/My Drive/dataset_final_3/A
/content/gdrive/My Drive/dataset_final_3/C
/content/gdrive/My Drive/dataset_final_3/D
/content/gdrive/My Drive/dataset_final_3/B
/content/gdrive/My Drive/dataset_final_3/W
/content/gdrive/My Drive/dataset_final_3/^
/content/gdrive/My Drive/dataset_final_3/E
/content/gdrive/My Drive/dataset_final_3/M
/content/gdri

Restart the terminal and execute every cell except the above one.

In [ ]:
vgg_conv = VGG16(weights = 'imagenet', include_top = False, input_shape = (input_size, input_size, 3))

for layer in vgg_conv.layers[:-4]:
  layer.trainable = False

for layer in vgg_conv.layers:
  print(layer, layer.trainable)
  
def createModel():
  model = models.Sequential()
  for layer in vgg_conv.layers:
    model.add(layer)
  model.add(layers.Flatten())
  model.add(layers.Dense(1024, activation = 'relu'))
  model.add(layers.Dropout(0.5))
  model.add(layers.Dense(noOfClasses, activation = 'softmax'))
  return model

def train():
  print("Loading data.....")
  X_train=np.load(path+"/X_train.npy")
  X_test=np.load(path+"/X_test.npy")
  y_train=np.load(path+"/y_train.npy")
  y_test=np.load(path+"/y_test.npy")
  model = createModel()
  print("Model summary")
  model.summary()
  batch_size = 10
  epochs = 8
  model.compile(optimizer = 'sgd', loss = 'categorical_crossentropy', metrics = ['accuracy'])
  print("Training........")
  model.fit(X_train, y_train, batch_size =batch_size, epochs = epochs, verbose = 1, validation_data = (X_test, y_test))
  print("Training complete!!!!")
  model.save(path+"/keras.FINAL_MODEL5")
train()

Instructions for updating:
Colocations handled automatically by placer.
58892288/58889256 [==============================] - 2s 0us/step
<keras.engine.topology.InputLayer object at 0x7f5406501be0> False
<keras.layers.convolutional.Conv2D object at 0x7f5406501eb8> False
<keras.layers.convolutional.Conv2D object at 0x7f5406501f98> False
<keras.layers.pooling.MaxPooling2D object at 0x7f54065080f0> False
<keras.layers.convolutional.Conv2D object at 0x7f54064a42e8> False
<keras.layers.convolutional.Conv2D object at 0x7f54064b5c88> False
<keras.layers.pooling.MaxPooling2D object at 0x7f54064d7588> False
<keras.layers.convolutional.Conv2D object at 0x7f540646b9b0> False
<keras.layers.convolutional.Conv2D object at 0x7f5406494128> False
<keras.layers.convolutional.Conv2D object at 0x7f5406494908> False
<keras.layers.pooling.MaxPooling2D object at 0x7f5406431ba8> False
<keras.layers.convolutional.Conv2D object at 0x7f54064583c8> False
<keras.layers.convolutional.Conv2D object at 0x7f54063f2b00>